In [ ]:
print(f"{u[-1]},{r[-1]}")
for _ in range(10):
    u.append(float(input()))
    r.append(float(input()))

In [20]:
with open("data.csv", "w") as f:
    for i in range(len(u)):
        f.write(f"{u[i]},{r[i]}\n")
        print(f"{u[i]},{r[i]}")

0.0,278.5
0.07,276.4
0.09,273.0
0.12,271.3
0.15,268.0
0.16,265.0
0.17,262.8
0.2,260.2
0.23,256.6
0.24,253.6
0.25,251.4
0.27,248.5
0.28,245.9
0.3,243.0
0.33,240.7
0.35,237.7
0.36,235.5
0.37,232.4
0.38,227.8
0.41,224.5
0.42,222.0
0.44,219.3
0.45,217.3
0.46,215.2
0.48,213.6
0.5,211.8
0.52,208.6
0.54,206.0
0.55,203.5
0.56,200.7
